In [ ]:
# 20 november 2025 - Sablina Vis / Dick van Mersbergen / Jonathan Blok
# Dit script voegt in bulk een nieuw veld toe aan bestaande pagina's op de Kennisbank

import sys
import logging
import requests
import getpass
import pandas as pd

logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, 
                    level=logging.INFO, 
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

try:
    excel_file_path = input('Documentpad (in de vorm /home/user/Documents/excel.xlsx): ')
    df = pd.read_excel(excel_file_path)
    df = df.reset_index()  # make sure indexes pair with number of rows

    base_url = input('API: ')
    username = input('Gebruikersnaam: ')
    password = getpass.getpass(prompt='Wachtwoord: ')
    kbank_field_name = input('Welk veld moet worden toegevoegd? (Zoals op de Kennisbank weergegeven, bv. "Gerelateerd aan gezicht" zonder quotes.): ')
    xls_field_name = input('In welk veld in de Excel sheet staat de data? (Bv. "gerelateerd_aan_gezicht" zonder quotes): ')

    # Initieren sessie met requests
    session = requests.Session()

    # Ophalen login token
    login_token_url = f"{base_url}?action=query&meta=tokens&type=login&format=json"
    login_token_response = session.get(login_token_url)
    login_token = login_token_response.json()["query"]["tokens"]["logintoken"]

    # Login
    login_url = f"{base_url}?action=login&format=json"
    login_params = {
        'lgtoken': login_token,
        'lgname': username,
        'lgpassword': password
    }

    login_response = session.post(login_url, data=login_params)
    login_result = login_response.json()["login"]["result"]
    logger.info("Login respons: %s", login_result)

    # Get the edit token
    edit_token_url = f"{base_url}?action=query&meta=tokens&type=csrf&format=json"
    edit_token_response = session.get(edit_token_url)
    edit_token = edit_token_response.json()["query"]["tokens"]["csrftoken"]

    for index, row in df.iterrows():
        ftext = f"{kbank_field_name}={row[xls_field_name]}"

        GET_PARAMS = {
            'action': "parse",
            'page': row['paginanaam'],
            'prop': 'wikitext',
            'format': "json"
        }

        response = session.get(base_url, params=GET_PARAMS)
        data = response.json()

        newcontents = [] # nieuwe content van de pagina

        if 'parse' not in data: # Geen pagina of data op pagina gevonden
            logger.warning("Pagina <%s> niet gevonden: <%s>", row['paginanaam'], data['error']['info'])
        else:
            wikitext = data['parse']['wikitext']['*']
            lines = wikitext.split('|')

            for line in lines:
                line_index = lines.index(line)
                if kbank_field_name in line: # staat het veld op deze regel?
                    if not line.split("=")[1].strip(): # is het veld leeg?
                        newcontents.append(ftext) # voeg veld met waarde toe aan nieuwe content
                        newcontents.extend(lines[line_index+1:]) # voeg de rest van de bestaande content aan nieuwe content toe
                        break
                    else: # als het veld niet leeg is geen wijziging doen 
                        newcontents = []
                        break
                elif "}}" in line and ("SourceDocument" in line or len(lines) == line_index+1): # einde van eerste template of SourceDocument bereikt zonder veld te vinden
                    newcontents.append(ftext) # voeg veld en nieuwe waarde toe aan nieuwe content 
                    newcontents.append(line) # voeg de rest van de content uit deze regel toe aan nieuwe content
                    newcontents.extend(lines[line_index+1:]) # voeg de rest van de bestaande content aan nieuwe content toe
                    break
                else:
                    newcontents.append(line) # voeg regel ongewijzigd toe aan nieuwe content

        if newcontents:
            EDIT_PARAMS = {
                "action": "edit",
                "title": row['paginanaam'],
                "format": "json",
                "token": edit_token,
                "text": "|".join(newcontents),
                "minor": True,
                "nocreate": True,
                "contentmodel": "wikitext",
                "format": "json"
            }

            edit_response = session.post(base_url, data=EDIT_PARAMS)
            edit_status = edit_response.json()["edit"]["result"]
        else:
            edit_status = "Pagina niet gewijzigd"

        logger.info("Edit status voor <%s> : %s", row['paginanaam'], edit_status)

except requests.JSONDecodeError as jsonde:
    logger.error("JSONDecoderError: mogelijk een fout bij authenticatie? ", {str(jsonde)})


2025-12-10 13:36:44 INFO     Login result: Success
2025-12-10 13:36:44 INFO     Edit status voor <Monumenten/403621> : Pagina niet gewijzigd
2025-12-10 13:36:44 INFO     Edit status voor <Monumenten/11558> : Pagina niet gewijzigd
2025-12-10 13:36:44 INFO     Edit status voor <Monumenten/11559> : Pagina niet gewijzigd
2025-12-10 13:36:45 INFO     Edit status voor <Monumenten/522928> : OK
2025-12-10 13:36:45 INFO     Edit status voor <Monumenten/523144> : Pagina niet gewijzigd
2025-12-10 13:36:45 INFO     Edit status voor <Monumenten/11551> : Pagina niet gewijzigd
2025-12-10 13:36:45 INFO     Edit status voor <Monumenten/11611> : OK
2025-12-10 13:36:45 INFO     Edit status voor <Monumenten/523137> : Pagina niet gewijzigd
2025-12-10 13:36:46 INFO     Edit status voor <Monumenten/523136> : Pagina niet gewijzigd
2025-12-10 13:36:46 INFO     Edit status voor <Monumenten/523138> : Pagina niet gewijzigd
2025-12-10 13:36:46 INFO     Edit status voor <Monumenten/523139> : Pagina niet gewijzigd
2